# Dataset Anthropic HH-RLHF

Take care of all the data cleaning to get the formatting to follow MT-Bench standards.

In [33]:
import pandas as pd
import random
import plotly as ply
import numpy as np
import re
import os
from tqdm import tqdm
import pickle

In [23]:
from datasets import load_dataset, load_dataset_builder

In [24]:
ds_builder = load_dataset_builder("anthropic/hh-rlhf")

In [25]:
ds_builder.info.description

''

In [26]:
ds_builder.info.features

{'chosen': Value(dtype='string', id=None),
 'rejected': Value(dtype='string', id=None)}

In [27]:
train_data = load_dataset("anthropic/hh-rlhf", split ='train[0:5000]')

Found cached dataset json (/Users/johnwang/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-a9fdd36e8b50b8fa/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [28]:
train_data

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 5000
})

In [29]:
df = train_data.to_pandas()

In [30]:
df.shape

(5000, 2)

In [36]:
# Combine 'chosen' and 'rejected' into a new column
df['combined'] = df.apply(lambda row: random.sample([row['chosen'], row['rejected']], 2), axis=1)

# Create a column to track the index of the 'chosen' string
df['chosen_index'] = df.apply(lambda row: 0 if row['chosen'] == row['combined'][0] else 1, axis=1)

# Reorder columns for better clarity
df = df[['combined', 'chosen_index']]

In [39]:
df = df.rename(columns={'combined':'conversations', 'chosen_index':'preference'})

# Create Human-Labeled Data JSONL: (hh_ldata.jsonl)

In [134]:
import json
with open("hh_human_labels.jsonl", "w") as file:
    for item in df.to_dict('records'):
        json_line = json.dumps(item)
        file.write(json_line + '\n')

# Create Prompt-Ready Data JSONL: (hh_questions.jsonl)

In [40]:
questions = df.copy()
questions = questions.drop('preference', axis='columns')

In [ ]:
import json
with open("hh_questions.jsonl", "w") as file:
    for item in questions.to_dict('records'):
        json_line = json.dumps(item)
        file.write(json_line + '\n')